## Libs

In [1]:
import os
import time
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import logging
from datetime import datetime

In [2]:
logging.captureWarnings(True)

## Define number of pages on each sections and saving path

In [8]:
NUMBER_OF_PAGES_ON_JUDICIAL = 417
NUMBER_OF_PAGES_ON_RECOVERY = 261

saving_path = f"../parsing_results/tazalau/{datetime.now().strftime('%Y%m%d')}"
# os.mkdir(saving_path)

In [9]:
def get_link_from_page(url: str, csv_file=saving_path):

    # Send HTTP GET request
    response = requests.get(url, verify=False)
    if response.status_code != 200:
        raise Exception(f"Failed to load page: {url}")

    # Parse HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find("table")
    # Step 3: Extract headers
    headers = []
    header_row = table.find("thead").find("tr")
    for th in header_row.find_all("th"):
        headers.append(th.get_text(strip=True))

    # Step 4: Extract rows
    rows = []
    for tr in table.find("tbody").find_all("tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]
        rows.append(cells)
    

    # Step 5: Create DataFrame
    df = pd.DataFrame(rows, columns=headers)
    
    return df
    



# get_link_from_page('https://tazalau.qoldau.kz/ru/list/bankruptcy/judicial?p=1')

## Recovery

In [10]:
page_numbers = range(1, NUMBER_OF_PAGES_ON_RECOVERY + 1)  # Pages 1 to last
df = pd.DataFrame()
for page in tqdm(page_numbers):
    # if page % 50 == 0:
    #     print(f'page {page} as successfully parsed')
    url = f"https://tazalau.qoldau.kz/ru/list/bankruptcy/recovery?p={page}"
    try:
        t_df = get_link_from_page(url, saving_path)
        df = pd.concat([df, t_df], ignore_index=True)
        time.sleep(1)  # Pause for 1 seconds between requests
    except Exception as e:
        print(f"Error on page {page}: {e}")
        time.sleep(5)  # Longer pause if error occurs
print(df.shape)
df.head()

100%|██████████| 261/261 [06:44<00:00,  1.55s/it]

(3906, 11)


,ФИО,Категория дела,Дата Иска,Дата передачи судье,Дата принятия к производству,Дата решения суда,Дата вступления в силу решения суда,Дата завершения дела,Наименование суда,Регион,Статус дела Torelik
0,АБДИРАМАНОВА М. Б.,Восстановление платежеспособности,16.10.2024,16.10.2024,,,08.11.2024,23.10.2024,Межрайонный суд по гражданским делам города Шы...,Шымкент,Иск заявление или жалоба возвращен
1,ДЖАНБАТИРОВ Б. Ш.,Восстановление платежеспособности,03.02.2025,03.02.2025,,,,07.02.2025,Илийский районный суд,Алматинская обл.,Исковое заявление возвращено
2,ТӨЛЕГЕН С. Б.,Восстановление платежеспособности,19.06.2025,19.06.2025,25.06.2025,,,,Атырауский городской суд,Атырауская обл.,Дело на рассмотрении
3,ОРМАНОВ Б. К.,Восстановление платежеспособности,29.10.2024,29.10.2024,,,20.11.2024,04.11.2024,Меркенский районный суд,Жамбылская обл.,Отказано в принятии искового заявления
4,ЖОЛДАСОВА М. М.,Восстановление платежеспособности,26.12.2024,26.12.2024,,,,05.01.2025,Сарысуский районный суд,Жамбылская обл.,Иск заявление или жалоба возвращен


In [11]:
df.head()

,ФИО,Категория дела,Дата Иска,Дата передачи судье,Дата принятия к производству,Дата решения суда,Дата вступления в силу решения суда,Дата завершения дела,Наименование суда,Регион,Статус дела Torelik
0,АБДИРАМАНОВА М. Б.,Восстановление платежеспособности,16.10.2024,16.10.2024,,,08.11.2024,23.10.2024,Межрайонный суд по гражданским делам города Шы...,Шымкент,Иск заявление или жалоба возвращен
1,ДЖАНБАТИРОВ Б. Ш.,Восстановление платежеспособности,03.02.2025,03.02.2025,,,,07.02.2025,Илийский районный суд,Алматинская обл.,Исковое заявление возвращено
2,ТӨЛЕГЕН С. Б.,Восстановление платежеспособности,19.06.2025,19.06.2025,25.06.2025,,,,Атырауский городской суд,Атырауская обл.,Дело на рассмотрении
3,ОРМАНОВ Б. К.,Восстановление платежеспособности,29.10.2024,29.10.2024,,,20.11.2024,04.11.2024,Меркенский районный суд,Жамбылская обл.,Отказано в принятии искового заявления
4,ЖОЛДАСОВА М. М.,Восстановление платежеспособности,26.12.2024,26.12.2024,,,,05.01.2025,Сарысуский районный суд,Жамбылская обл.,Иск заявление или жалоба возвращен


In [12]:


start = datetime.now().strftime('%Y%m%d')
start

'20250701'

In [13]:

df.to_excel(f'{saving_path}/bankruptcy_recovery.xlsx', index=False)

## Judicial

In [26]:
page_numbers = range(1, NUMBER_OF_PAGES_ON_JUDICIAL + 1)  # Pages 1 to last
df = pd.DataFrame()
for page in tqdm(page_numbers):
    # if page % 50 == 0:
    #     print(f'page {page} as successfully parsed')
    url = f"https://tazalau.qoldau.kz/ru/list/bankruptcy/judicial?p={page}"
    try:
        t_df = get_link_from_page(url, saving_path)
        df = pd.concat([df, t_df], ignore_index=True)
        time.sleep(0.5)  # Pause for 1 seconds between requests
    except Exception as e:
        print(f"Error on page {page}: {e}")
        time.sleep(5)  # Longer pause if error occurs
print(df.shape)
df.head()

100%|██████████| 417/417 [07:08<00:00,  1.03s/it]

(6251, 13)


,ИИН,ФИО,Категория дела,Дата Иска,Дата передачи судье,Дата принятия к производству,Дата решения суда,Дата вступления в силу решения суда,Дата завершения дела,Наименование суда,Регион,Статус дела Torelik,Статус дела Tazalau
0,660615301860,РЫЛЬСКИЙ А. В.,Банкротство физ. лиц,19.02.2024,19.02.2024,,,13.03.2024,23.02.2024,Риддерский городской суд,Восточно-Казахстанская обл.,Исковое заявление возвращено,"Процедура прекращена (отказано, иск возвращен,..."
1,771102302032,ЦАЙ А. Э.,Банкротство физ. лиц,05.06.2023,05.06.2023,,,03.07.2023,12.06.2023,Кызылординский городской суд,Кызылординская обл.,Исковое заявление возвращено,"Процедура прекращена (отказано, иск возвращен,..."
2,861017401088,ТАЖИБЕКОВА Б. С.,Банкротство физ. лиц,06.06.2023,06.06.2023,,,23.06.2023,11.06.2023,Наурызбайский районный суд,Алматы,Иск заявление или жалоба возвращен,"Процедура прекращена (отказано, иск возвращен,..."
3,810312401921,КАНАФИНА Д. А.,Банкротство физ. лиц,12.05.2023,12.05.2023,,,31.05.2023,17.05.2023,Бостандыкский районный суд,Алматы,Исковое заявление возвращено,"Процедура прекращена (отказано, иск возвращен,..."
4,930730302287,КУЛИКОВ Б. В.,Банкротство физ. лиц,22.05.2025,22.05.2025,29.05.2025,,,,Турксибский районный суд,Алматы,Дело на рассмотрении,На рассмотрении


In [27]:
df.to_excel(f'{saving_path}/bankruptcy_judicial.xlsx', index=False)